In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS 
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEndpoint
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

In [2]:
import os 
from dotenv import load_dotenv

load_dotenv()

os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.getenv("HUGGINGFACE_API_KEY")
repo_id = "mistralai/Mistral-7B-Instruct-v0.2"
hf_endpoint = HuggingFaceEndpoint(
        repo_id=repo_id, 
        temperature=0.7, 
        model_kwargs={
            "max_length": 128,
            "token": os.getenv("HUGGINGFACE_API_KEY")
        }
)

d:\Artificial Intelligence\Langchain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\praja\.cache\huggingface\token
Login successful


In [3]:
## Reading PDF
loader = PyPDFDirectoryLoader("./us_census")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
final_documents = text_splitter.split_documents(documents)
final_documents[0]

Document(metadata={'source': 'us_census\\acsbr-015.pdf', 'page': 0}, page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015Issued September 2023Douglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to health coverage. For example, between 2021 and 2022, the labor market continued to improve, which may have affected private coverage in the United States \nduring that time.\n1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nallowed Medicaid enrollees to remain covered under the Continuous Enrollment Provision.\n2 The American \nRescue Plan (ARP) enhanced Marketplace premium subsidies for those with incomes above 400 percent of the poverty level as well as for unemployed people.\n3')

In [4]:
len(final_documents)

316

In [5]:
## Embedding using huggingface
huggingface_embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    model_kwargs = {'device':'cpu'},
    encode_kwargs = {'normalize_embeddings': True}
)

d:\Artificial Intelligence\Langchain\venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [6]:
import numpy as np
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)))
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)).shape)


[-7.81247616e-02 -1.76957175e-02 -2.55395658e-02  3.16485763e-02
  4.89566736e-02  5.73916174e-02 -3.41920033e-02  2.61430740e-02
 -9.73034576e-02 -2.28453092e-02  7.35054389e-02  5.15418313e-02
 -1.08876415e-02 -3.11451592e-02  3.21608013e-03  4.89183404e-02
 -1.29311113e-02 -1.42324355e-03 -3.19031030e-02  2.54794918e-02
 -3.13170627e-02  3.35001610e-02 -2.71926597e-02 -4.65959162e-02
  1.61344819e-02  1.42099615e-02 -7.43198767e-03  1.13745006e-02
 -5.09818681e-02 -1.41039535e-01  4.36505163e-03  2.54208110e-02
 -5.71403578e-02 -1.06509784e-02  2.70821042e-02  1.18388301e-02
 -2.27215737e-02  7.71624669e-02  4.27384116e-02  6.18330874e-02
 -3.68240029e-02  4.53973282e-03 -2.01893449e-02 -4.70244093e-03
 -2.41782796e-02 -5.31012702e-05 -3.19753811e-02 -4.51394590e-03
  3.41201713e-03 -5.89688532e-02  2.63294522e-02 -8.91791284e-03
  5.06487861e-02  6.75783232e-02  6.58879504e-02 -4.71076407e-02
  2.60821413e-02 -2.50376314e-02 -4.24258560e-02  3.94773558e-02
  4.30341326e-02 -3.05914

In [7]:
vectorstore=FAISS.from_documents(final_documents[:50],huggingface_embeddings)

In [8]:
## Query using Similarity Search
query="WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [9]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001B9D44A0700> search_kwargs={'k': 3}


In [10]:
query = "What is the health insurance coverage?"
hf_endpoint.invoke(query)

"\n\nThe health insurance coverage for each person may vary depending on their individual circumstances, including their age, health status, income, and employment situation. Here are some common types of health insurance coverage:\n\n1. Employer-sponsored insurance: Many employers offer health insurance coverage as a benefit to their employees. This type of insurance is typically paid for in part or in full by the employer, and the employee may be required to contribute a portion of the premium through payroll deductions.\n2. Individual and family plans: Individuals and families can purchase health insurance coverage on their own through the Affordable Care Act (ACA) marketplace or from private insurers. These plans may be more expensive than employer-sponsored insurance, and the cost and availability of coverage may vary depending on the individual's location and health status.\n3. Medicare: Medicare is a federal health insurance program for people over the age of 65, as well as some

In [11]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """


In [12]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [13]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf_endpoint,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [14]:

query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [15]:

# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])

1. In 2022, the uninsured rate was lower in states that had expanded Medicaid eligibility than in states that did not. Specifically, the uninsured rate was 22.7% in expansion states and 18.0% in nonexpansion states.
 2. From 2021 to 2022, 27 states saw a decrease in their uninsured rates, while only Maine had an increase. Maine's uninsured rate increased from 5.7% to 6.6%, but remained below the national average.
 3. Massachusetts had the lowest uninsured rate in 2022, while Texas had the highest.
 4. Utah had the highest private coverage and lowest public coverage rate in 2022, while New Mexico had the highest public coverage and lowest private coverage rate.
 5. There were no statistically significant differences in private coverage rates between North Dakota and Utah in 2022.
 6. The American Community Survey is a source of information on the uninsured rate and coverage types by state and metropolitan area.
